In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

jtplot.style(figsize=(15, 9))

In [2]:
map_data = pd.read_csv('../data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [24]:
useless_clmns = ['FileDate', 'Source', 'ActionType', 'Ori', 'Subcircum', 'Solved']
yes = map_data[(map_data['Solved']=='Yes')
               & ~(map_data['OffSex']=='Unknown')
               & ~(map_data['OffRace']=='Unknown')
              ].drop(useless_clmns, axis=1)
no = map_data[map_data['Solved']=='No'].drop(useless_clmns, axis=1) # test

In [25]:
yes.shape, no.shape

((602887, 23), (256868, 23))

In [20]:
# Relationship not determined / Circumstances undetermined

In [26]:
yes.head()

,CNTYFIPS,State,Agency,Agentype,Year,Month,Incident,Homicide,Situation,VicAge,...,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,VicCount,OffCount,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",Alaska,State Troopers,Primary state LE,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,48,...,55,Female,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,0,0,Rural Alaska
197601001AL00102,"Jefferson, AL",Alabama,Birmingham,Municipal police,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,65,...,67,Male,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,0,0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",Alabama,Fairfield,Municipal police,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,45,...,53,Male,Black,Unknown or not reported,Shotgun,Wife,Other,0,0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",Alabama,Leeds,Municipal police,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,43,...,35,Female,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,0,0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",Alabama,Mobile,Municipal police,1976,January,1,Murder and non-negligent manslaughter,Single victim/single offender,35,...,25,Female,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,0,0,"Mobile, AL"


In [27]:
yes.dtypes

CNTYFIPS        object
State           object
Agency          object
Agentype        object
Year             int64
Month           object
Incident         int64
Homicide        object
Situation       object
VicAge           int64
VicSex          object
VicRace         object
VicEthnic       object
OffAge           int64
OffSex          object
OffRace         object
OffEthnic       object
Weapon          object
Relationship    object
Circumstance    object
VicCount         int64
OffCount         int64
MSA             object
dtype: object